In [2]:
import openai

import configparser

config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['openai']['api_key']
openai.api_key = api_key

In [3]:
# Define a function to interact with the ChatGPT model
def chat_with_model(message):
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=message,
        max_tokens=50,
        temperature=0.7,
        n=1,
        stop=None,
        timeout=10
    )
    return response.choices[0].text.strip()


In [4]:
chat_with_model("Hello, I'm a robot")

'Nice to meet you!'

### Intro langchain

In [5]:
from langchain.llms import OpenAI

In [12]:
llm = OpenAI(model_name='text-davinci-003', temperature=0.3)

In [13]:
comment = """
Vota mucho gas y emana olores, la compre pensando que iba a ser un ahorro en la luz, pero fue todo lo contrario, consumio un balon de gas premiun en una semana. Encima la valvula vino desconpuesta por dentro, al agitar se escuchaba que tenia algo adentro, tampoco ajustaba bien al balon de gas, quedaba muy suelto. Tuve que devolverla.
"""

In [20]:
from langchain import PromptTemplate

template = """ 
Según el comentario, cuáles son las principales características del producto?

Comentario: {comment}
"""

prompt = PromptTemplate(template = template, 
                        input_variables=['comment'])

In [21]:
prompt.format(comment=comment)

' \nSegún el comentario, cuáles son las principales características del producto?\n\nComentario: \nVota mucho gas y emana olores, la compre pensando que iba a ser un ahorro en la luz, pero fue todo lo contrario, consumio un balon de gas premiun en una semana. Encima la valvula vino desconpuesta por dentro, al agitar se escuchaba que tenia algo adentro, tampoco ajustaba bien al balon de gas, quedaba muy suelto. Tuve que devolverla.\n\n'

In [22]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run(comment))


Las principales características del producto son: consumo excesivo de gas, emisión de olores, válvula defectuosa y ajuste inadecuado al balón de gas.


### Few shots

In [23]:
from langchain import FewShotPromptTemplate

examples = [{"comentario": "La pelota es muy liviana", "palabras clave": ["liviana"]},
            {"comentario": "El jabón tiene un olor exquisito", "palabras clave": ["fragancia exquisita"]}]

example_formatter = """
Comentario: {comentario}
Palabras clave: {palabras clave}\n
"""

example_prompt = PromptTemplate(input_variables=['comentario', 'palabras clave'],
                                template=example_formatter)


In [24]:
few_shot_prompt = FewShotPromptTemplate(examples = examples,
                                        example_prompt = example_prompt,
                                        prefix= "Según el comentario, cuáles son las principales características del producto?\n",
                                        suffix= "\n\nComentario: {input}\nPalabras clave:",
                                        input_variables=['input'],
                                        example_separator="\n\n")

In [25]:
print(few_shot_prompt.format(input=comment))

Según el comentario, cuáles son las principales características del producto?



Comentario: La pelota es muy liviana
Palabras clave: liviana




Comentario: El jabón tiene un olor exquisito
Palabras clave: fragancia exquisita





Comentario: 
Vota mucho gas y emana olores, la compre pensando que iba a ser un ahorro en la luz, pero fue todo lo contrario, consumio un balon de gas premiun en una semana. Encima la valvula vino desconpuesta por dentro, al agitar se escuchaba que tenia algo adentro, tampoco ajustaba bien al balon de gas, quedaba muy suelto. Tuve que devolverla.

Palabras clave:


In [26]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt)

print(chain.run(comment))

 Consumo de gas alto, valvula descompuesta, ajuste suelto.


### Add chains

In [ ]:
from langchain.chains import SimpleSequentialChain, SequentialChain

full_chain = SequentialChain(chains = [chain, chain, chain],verbose=True)

print(full_chain.run(comment))

### Chat llm chain

In [ ]:
from langchain.llms import OpenAIChat

prefix_message = [{"role":"system","content":"You are a helpful assistant"}]


llm = OpenAIChat(model_name='gpt-3.5-turbp', temperature=0.3, prefix_message=prefix_message)

## Embeddings

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

### Next line is reference

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, overlap_size=200)
#texts = text_splitter.split(documents)

In [15]:
import json

with open('data.json') as f:
    sample_response = json.load(f)

In [16]:
def product_text(product):
    text = f"""
    title: {product['title']}
    description: {product['description']}
    specs: {", ".join([x + " " + y for x,y in product['specs'].items()])}
    """
    return text

In [17]:
texts = [product_text(product) for product in sample_response]

In [18]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db2'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_texts(texts=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [20]:
vectordb.persist()
vectordb = None

In [21]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

In [22]:
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

In [ ]:
demo_aspiradora = 

In [14]:
retriever.get_relevant_documents("aspiradora barata")

[Document(page_content='\n    title: Aspiradora xd\n    description: aspiradora de mano\n    specs: Corriente 5A, color rojo\n    ', metadata={})]